In [1]:
#Import the corresponding libraries
import pandas as pd
import pickle
import os

In [2]:
#Read Apache Parquet DataSet
'''
Each of the files for this project contains 2 tabs. For instance:
The file completo800_(edit).xlsx contains the tabs:

* totale
* Foglio 1

For performance reasons the original .xlsx files were converted to Apache_Parquet files
After this transformation the only remainig tab is

* totale

which contains all the data concerning to the research
''' 
#DataFrame used for deployment
# '/Users/luisescobar/Documents/Thesis/DataSets/Apache_parquet/completo_800_output_file.parquet'

path_to_read = '/Users/luisescobar/Documents/Thesis/DataSets/Apache_parquet'
path_to_save = '/Users/luisescobar/Documents/Thesis/DataSets/Dictionary'
file = 'completo_800_output_file.parquet'
name_read = f'{path_to_read}/{file}'
df = pd.read_parquet(name_read, engine='fastparquet')

In [3]:
df.head()

,Patient,Date__Heure,P_Access,P_Filter,P_Effluent,P_Return,FIFTH_PRESSURE,RUN_TIME,Post_Replacement,Pre_Replacement,...,Citric_Acid_Concentration__mmol_,Citrate_Dose__mmol_L_,Calcium_Compensation____,Calcium_syringe_Concentration__m,Treatment_Duration__h_,effluent2,delta_eff,max_line,reverse,sum
0,NaN,2012-05-12 21:41:00,-9,60,21,23,-10,78951,12731,0,...,_,_,_,_,19.098612,49518,-40,2608,1156,-41843
1,NaN,2012-05-12 21:42:00,-14,88,24,32,-10,79011,12741,0,...,_,_,_,_,19.098612,49557,-39,2608,1155,-41803
2,NaN,2012-05-12 21:43:00,-16,125,66,61,-9,79071,12751,0,...,_,_,_,_,19.098612,49595,-38,2608,1154,-41764
3,NaN,2012-05-12 21:44:00,-13,123,66,63,-9,79131,12762,0,...,_,_,_,_,19.098612,49634,-39,2608,1153,-41726
4,NaN,2012-05-12 21:45:00,-15,117,58,58,-9,79191,12771,0,...,_,_,_,_,19.098612,49673,-39,2608,1152,-41687


In [4]:
#Info of the DataFrame
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 311232 entries, 0 to 311231
Data columns (total 94 columns):
 #   Column                            Non-Null Count   Dtype         
---  ------                            --------------   -----         
 0   Patient                           0 non-null       float64       
 1   Date__Heure                       311232 non-null  datetime64[ns]
 2   P_Access                          311232 non-null  int64         
 3   P_Filter                          311232 non-null  int64         
 4   P_Effluent                        311232 non-null  int64         
 5   P_Return                          311232 non-null  int64         
 6   FIFTH_PRESSURE                    311232 non-null  int64         
 7   RUN_TIME                          311232 non-null  int64         
 8   Post_Replacement                  311232 non-null  int64         
 9   Pre_Replacement                   311232 non-null  int64         
 10  Dialysate                       

In [5]:
def dict_trt(df_spec_col):
    '''The function returns a dictionary with keys associated to
    the different treatment 'trt' values
    Parameters
    ----------
    df_spec_col : pandas.core.frame.DataFrame
        DataFrame with specific columns related to the research focus 

        
    Returns
    -------
    df_dict : dict
        Dictionary with DataFrames divided according to the treatment 'trt' value.
        The output of dict.keys will be something as follows
            dict_keys(['df_601', 'df_602', 'df_603', 'df_604', 'df_605', ...
        df_601 is by itself a DataFrame with all the information corresponding to the
        treatment 'trt' 601
    '''
    
    #List with the different treatments values
    trt = df_spec_col["trt"].unique()
    df_dict = {}

    for value in trt:
        # Create a dataframe for each unique value in trt and store it in the dictionary
        df_dict[f"df_{value}"] = df_spec_col[df_spec_col['trt'] == value]
        
    return df_dict


In [6]:
def trt_weight_vis(df_dict):
    '''The function returns a sub DataFrame with the columns 
    trt and Patient_weight__Kg_
    Parameters
    ----------
    df_dict : dict
        Dictionary organized according to the different treatments 'trt'
        values

        
    Returns
    -------
    df : pandas.core.frame.DataFrame
        DataFrame with 'weight' set as index and 'trt' as reference value 
        the 'weight' (index) is sorted ascending
        
                    
        weight(index)----trt
        7.0--------------706
        7.0--------------702
        45.0-------------616
        45.0-------------613
        46.0-------------618

    '''
    merge_dict = {
        'trt':[],
        'weight':[]
    }
    for value in df_dict.keys():
        df = df_dict[value]
        trt = df["trt"].unique()
        weight = df["Patient_weight__Kg_"].unique()
        merge_dict['trt'].extend(trt)
        merge_dict['weight'].extend(weight)
        
    df = pd.DataFrame(merge_dict)
    df = df.sort_values(by=['weight', 'trt']).reset_index(drop=True)
    df = df.set_index("weight", inplace=False)
    
    return df

<div class="alert alert-block alert-warning">
  <p>
    <b>To Do</b>
  </p>

  <p>
    I would like to have a function in which I choose, arbitrary, the maximum difference, between trt values, and according to this merge the DataFrames
  </p>
    
  <p>
  </p>
</div>

In [7]:
def trt_to_merge(df, tolerance):
    '''The function returns a DataFrame with boolean information, in the column merge, to 
    join DataFrames that correspond to the same patient
    
    Parameters
    ----------
    df : pandas.core.frame.DataFrame
        DataFrame containing just the 'trt' and 'weight' value for all the cases of interest. Output of the 
        trt_weight_vis function
        
    tolerance : int
        Tolerance of difference between the treatment values corresponding to a patient with the 
        same weight
        i.e., a tolerance of 1 indicates that given the follwing DataFrame
        
            weight   trt    diff    tolerance   merge
        [0] 7.0      702    4.0     False       False
        [1] 7.0      706    93.0    False       False
        [2] 45.0     613    3.0     False       False
        [3] 45.0     616    2.0     False       False
        [4] 46.0     618    1.0     True        True
        [5] 46.0     619    7.0     False       False
        
        The row [4] will be True since, is the only one in which, the difference between trt[i] and 
        trt[i+1] is 1
        
    Returns
    -------
    df : pandas.core.frame.DataFrame
        DataFrame with the following columns
        ['weight'   'trt'    'diff'    'tolerance'   'merge']
        
        weight(index): weight value of the different patients
        trt: treatment value
        diff: difference between consecutive treatment values after sorting the DataFrame
        tolerance:  if diff <= tolerance
                        return True
                    else
                        return False
        merge:  if (df['tolerance'] == True) & (shifted_index == df.index)
                    return True
                else
                    return True
        
        shifted_index is the value of the index in the position [i+1] 
        
        where: 
            i is the current index position 
                    
    '''
    # Shifting the 'trt' column by one position
    df['shifted_trt'] = df['trt'].shift(-1)

    # Calculating the difference between the 'trt' column and the shifted 'trt' column
    df['diff'] = abs(df['trt'] - df['shifted_trt'])

    # Dropping the 'shifted_trt' column as it's no longer needed
    df.drop(columns=['shifted_trt'], inplace=True)
    
    #Search values in which the diff is <= tolerance
    df['tolerance'] = df['diff'] <= tolerance
    
    #Create a new column merge to do the final comparison between trt values and weight values
    shifted_index = df.index.to_series().shift(-1)
    df['merge'] = ((df['tolerance'] == True) & (shifted_index == df.index))

    
    return df

In [8]:
def patient_recognition(df, columns, tolerance):
    '''The function returns a DataFrame with columns containing information
    useful to match patients with 'trt' number. This is a global function, cointaining all the 
    previous defined functions.
    Parameters
    ----------
    df : pandas.core.frame.DataFrame
        DataFrame with all the patients information
    columns : list
        Columns of interest i.e.,
        columns=["Date__Heure","P_Access","P_Filter","P_Effluent","P_Return", ...
    tolerance : int
        Tolerance of difference between the treatment values corresponding to a patient with the 
        same weight
        i.e., a tolerance of 1 indicates that given the follwing DataFrame
        
            weight   trt    diff    tolerance   merge
        [0] 7.0      702    4.0     False       False
        [1] 7.0      706    93.0    False       False
        [2] 45.0     613    3.0     False       False
        [3] 45.0     616    2.0     False       False
        [4] 46.0     618    1.0     True        True
        [5] 46.0     619    7.0     False       False
        
        The value [4] will be True since, is the only one in which, the difference between trt[i] and 
        trt[i+1] is 1 

        
    Returns
    -------
    df : pandas.core.frame.DataFrame
        DataFrame with the following columns
        ['weight'   'trt'    'diff'    'tolerance'   'merge']
        weight(index): weight value of the different patients
        trt: treatment value
        diff: difference between consecutive treatment values after sorting the DataFrame
        tolerance:  if diff <= tolerance
                        return True
                    else
                        return False
        merge:  if (df['tolerance'] == True) & (shifted_index == df.index)
                    return True
                else
                    return True
        
        shifted_index is the value of the index in the position [i+1] 
        
        where: 
            i is the current index position 
        
    '''
    #Extract specific columns from the original DataFrame
    df_spec_col = df[columns]
    #Dictionary in which the indexes are the different trt values
    df_dict = dict_trt(df_spec_col)
    #DataFrame to evaluate weight with trt
    df = trt_weight_vis(df_dict)
    #DataFrame to evaluate which trt should be merged 
    df = trt_to_merge(df, tolerance)
    
    return df,df_dict

<div class="alert alert-block alert-info"> <b>Part One</b> 
    <p>
        <ul>
        <li>DataFrame ['df_patients'] with information of the possible trt values to merge. This decision depends on the diff value; tolerance parameter.  </li>
        <li>Dictionary ['df_dict_initial'] in which each key corresponds to the DataFrame of the respective trt number. The dictionary is a collection of DataFrames.</li>
        </ul>
             </p> </div>

In [9]:
#Columns to consider from the original Data
columns=["Date__Heure","P_Access","P_Filter","P_Effluent","P_Return","Q_Blood_Pump",
          "Q_Replacement", "Q_Dialysate", "Q_PBP", "Q_Patient_Fluid_Removal", "DeltaP", "TMP", "TMPa", "trt", 
         "Patient_weight__Kg_", "Set"]
df_patients, df_dict_initial = patient_recognition(df,columns,1)

In [10]:
df_patients.head(10)

,trt,diff,tolerance,merge
weight,,,,
7.0,702,4.0,False,False
7.0,706,93.0,False,False
45.0,613,3.0,False,False
45.0,616,2.0,False,False
46.0,618,1.0,True,True
46.0,619,7.0,False,False
46.0,626,23.0,False,False
50.0,603,9.0,False,False
50.0,612,56.0,False,False


<div class="alert alert-block alert-info"> <b>Part Two</b> 
    <p>
        Dictionary ['dictionary_merge']  with the information of the trt to be merged. Form the previous DataFrame ['df_patients'], if merge == True, we join the current [i] and the next trt [i+1] as treatments to be merged
    </p> 
</div>

In [11]:
def dictionary_merge(df_patients):
    '''The function returns a dictionary with the pair of treatments to merge
    Parameters
    ----------
    df_patients : pandas.core.frame.DataFrame
        DataFrame with the columns
         weight   trt    diff    tolerance   merge

    Returns
    -------
    trt_merge_dict : dict
        Dictionary with tuples corresponding to the trt values to merge
        
    '''
    df = df_patients.reset_index()
    indexes = df[df['merge'] == True].index.tolist()
    trt_merge_dict = {}
    
    for i in range(len(indexes)):  
        index_val = indexes[i]
        trt_val = df['trt'][index_val]
        dict_name = f"index_{index_val}"
        new_trt_merge = {'index': index_val, 'trt': {trt_val,trt_val+1}}
        trt_merge_dict.update({dict_name: new_trt_merge})
    
    #Sort dict according to 'trt'
    sorted_data = dict(sorted(trt_merge_dict.items(), key=lambda item: min(item[1]['trt'])))
    
    # Convert 'trt' sets to sorted lists
    for key in sorted_data:
        sorted_data[key]['trt'] = sorted(sorted_data[key]['trt'])
        
    return sorted_data
        

In [12]:
trt_merge_dict = dictionary_merge(df_patients)
trt_merge_dict

{'index_48': {'index': 48, 'trt': [608, 609]},
 'index_4': {'index': 4, 'trt': [618, 619]},
 'index_16': {'index': 16, 'trt': [624, 625]},
 'index_66': {'index': 66, 'trt': [632, 633]},
 'index_90': {'index': 90, 'trt': [649, 650]},
 'index_116': {'index': 116, 'trt': [660, 661]},
 'index_119': {'index': 119, 'trt': [680, 681]},
 'index_154': {'index': 154, 'trt': [682, 683]},
 'index_141': {'index': 141, 'trt': [684, 685]},
 'index_142': {'index': 142, 'trt': [685, 686]},
 'index_63': {'index': 63, 'trt': [687, 688]},
 'index_145': {'index': 145, 'trt': [695, 696]},
 'index_146': {'index': 146, 'trt': [696, 697]},
 'index_24': {'index': 24, 'trt': [698, 699]},
 'index_175': {'index': 175, 'trt': [712, 713]},
 'index_72': {'index': 72, 'trt': [718, 719]},
 'index_96': {'index': 96, 'trt': [720, 721]},
 'index_98': {'index': 98, 'trt': [723, 724]},
 'index_161': {'index': 161, 'trt': [737, 738]},
 'index_162': {'index': 162, 'trt': [738, 739]},
 'index_36': {'index': 36, 'trt': [744, 74

In [13]:
# Convert the dictionary to a DataFrame
df_dict = pd.DataFrame.from_dict(trt_merge_dict, orient='index')

# Reset the index to have a default integer index and move the original index to a column
df_dict.reset_index(inplace=True)

# Rename the column for clarity
df_dict.rename(columns={'index': 'original_index'}, inplace=True)
df_dict.head(15)

,level_0,original_index,trt
0,index_48,48,"[608, 609]"
1,index_4,4,"[618, 619]"
2,index_16,16,"[624, 625]"
3,index_66,66,"[632, 633]"
4,index_90,90,"[649, 650]"
5,index_116,116,"[660, 661]"
6,index_119,119,"[680, 681]"
7,index_154,154,"[682, 683]"
8,index_141,141,"[684, 685]"
9,index_142,142,"[685, 686]"


<div class="alert alert-block alert-warning"> 
    At this point we found the problem that in some cases we have to merge more than 2 treatments. Consider as example, the iloc indexes, [8] and [9]
    <li>[8] corresponds to the treatments [684, 685] which are treatments corresponding to the same patient, beacause of the weight value</li>
    <li>[9] corresponds to the treatments [685, 686] which are treatments corresponding to the same patient, beacause of the weight value</li>
    We have an intersection at the value 685, this means that treatments 684, 685, 686 potentially corresponds to the same patient. 
</div>

In [14]:
def trt_compar(df, i):
    '''The function compares the trt values looking for an intersection between two continuous rows. i.e., [0] and [1] 
    Parameters
    ----------
    df : pandas.core.frame.DataFrame
        DataFrame with the columns

        level_0	    original_index	    trt
    [0]	index_48	     48	         [608, 609]
    [1]	index_4	          4	         [618, 619]
    [2]	index_16	     16	         [624, 625]

    i : iterator to go through the different rows of the DataFrame      
    Returns
    -------
    True / False : boolean
        Boolean with the result of the comparisson
        
    '''
    value_01 = list(df_dict['trt'].iloc[i])[1]
    value_02 = list(df_dict['trt'].iloc[i+1])[0]
    if value_01 == value_02:
        return True
    else:
        return False

In [15]:
def dictionary_multiple_merge(df_dict):
    '''The function returns a dictionary with the list of all the merges we could potentially do. 
    This information was obtained according to the trt value and the patient's weight.
  
    Parameters
    ----------
    df_dict : pandas.core.frame.DataFrame
        DataFrame with the columns

        level_0	    original_index	    trt
    [0]	index_48	     48	         [608, 609]
    [1]	index_4	          4	         [618, 619]
    [2]	index_16	     16	         [624, 625]
      
    Returns
    -------
    dict : dict
        Dictionary with the potential treatments to merge. We expect something like the following output
        
        {'index_48': {'values': [608, 609]},
         'index_4': {'values': [618, 619]},
         'index_16': {'values': [624, 625]},
         'index_66': {'values': [632, 633]},
         'index_90': {'values': [649, 650]},
         'index_116': {'values': [660, 661]},
         'index_119': {'values': [680, 681]},
         'index_154': {'values': [682, 683]},
         'index_142': {'values': ([684, 685], [685, 686])}, ...
        
    '''
    dict={}
    i=0
    iter_range = len(df_dict)-1
    #Given a DataFrame like the one from the example, we iterate through the whole list of trt values ['iter_range']
    while i < iter_range:
        result = trt_compar(df_dict,i)
        list_trt = []
        if result == True:
            #Save the first tuple in a list
            list_trt.append((df_dict['trt'].iloc[i],
                       (df_dict['trt'].iloc[i+1])))
            #Search more tuples to save in the list
            for j in range(i+1,iter_range):
                result = trt_compar(df_dict,j)
                if result == True:
                    list_trt.append([df_dict['trt'].iloc[j+1]])
                else:
                    diff = j-i
                    i += diff
                    break    
    
            list_name = f"index_{df_dict['original_index'].iloc[i]}"
            list_merge = {'values': list_trt}
            dict.update({list_name: list_merge})        
        
        else:
            list_trt = list(df_dict['trt'].iloc[i])
            list_name = f"index_{df_dict['original_index'].iloc[i]}"
            list_merge = {'values': list_trt}
            dict.update({list_name: list_merge})
        i += 1
    return dict

In [16]:
dict = dictionary_multiple_merge(df_dict)

In [17]:
dict

{'index_48': {'values': [608, 609]},
 'index_4': {'values': [618, 619]},
 'index_16': {'values': [624, 625]},
 'index_66': {'values': [632, 633]},
 'index_90': {'values': [649, 650]},
 'index_116': {'values': [660, 661]},
 'index_119': {'values': [680, 681]},
 'index_154': {'values': [682, 683]},
 'index_142': {'values': [([684, 685], [685, 686])]},
 'index_63': {'values': [687, 688]},
 'index_146': {'values': [([695, 696], [696, 697])]},
 'index_24': {'values': [698, 699]},
 'index_175': {'values': [712, 713]},
 'index_72': {'values': [718, 719]},
 'index_96': {'values': [720, 721]},
 'index_98': {'values': [723, 724]},
 'index_162': {'values': [([737, 738], [738, 739])]},
 'index_37': {'values': [([744, 745], [745, 746])]},
 'index_164': {'values': [749, 750]},
 'index_167': {'values': [([752, 753], [753, 754])]},
 'index_41': {'values': [([759, 760], [760, 761])]},
 'index_56': {'values': [765, 766]},
 'index_170': {'values': [([767, 768], [768, 769])]},
 'index_105': {'values': [77

In [18]:
dict.items()

dict_items([('index_48', {'values': [608, 609]}), ('index_4', {'values': [618, 619]}), ('index_16', {'values': [624, 625]}), ('index_66', {'values': [632, 633]}), ('index_90', {'values': [649, 650]}), ('index_116', {'values': [660, 661]}), ('index_119', {'values': [680, 681]}), ('index_154', {'values': [682, 683]}), ('index_142', {'values': [([684, 685], [685, 686])]}), ('index_63', {'values': [687, 688]}), ('index_146', {'values': [([695, 696], [696, 697])]}), ('index_24', {'values': [698, 699]}), ('index_175', {'values': [712, 713]}), ('index_72', {'values': [718, 719]}), ('index_96', {'values': [720, 721]}), ('index_98', {'values': [723, 724]}), ('index_162', {'values': [([737, 738], [738, 739])]}), ('index_37', {'values': [([744, 745], [745, 746])]}), ('index_164', {'values': [749, 750]}), ('index_167', {'values': [([752, 753], [753, 754])]}), ('index_41', {'values': [([759, 760], [760, 761])]}), ('index_56', {'values': [765, 766]}), ('index_170', {'values': [([767, 768], [768, 769

In [19]:
def flatten_list(nested_list):
    """Recursively flatten a nested list or tuple."""
    flat_list = []
    for item in nested_list:
        if isinstance(item, (list, tuple)):
            flat_list.extend(flatten_list(item))
        else:
            flat_list.append(item)
    return flat_list

def remove_duplicates(input_list):
    """Remove duplicates from a list while maintaining order."""
    unique_list = []
    seen = set()
    for item in input_list:
        if item not in seen:
            unique_list.append(item)
            seen.add(item)
    return unique_list


In [20]:
# We create the final dictionary. Each key is a DataFrame; at the end we appended those corresponding to potentially merges

for key, value in dict.items():
    values = value['values']
    # If values is a tuple, combine the inner lists into a single list
    if isinstance(values, tuple):
        combined_values = []
        for sublist in values:
            combined_values.extend(sublist)
        values = combined_values
    # Remove duplicates from the list
    values = flatten_list(values)
    values = remove_duplicates(values)
    # Add prefix "df_" to each element in the list
    values = [f"df_{v}" for v in values]
    #Merge the DataFrames
    concat_df = pd.concat([df_dict_initial[key] for key in values], axis=0)
    values_str = '-'.join(values)
    name = f'{values_str}-concat'
    df_dict_initial[name] = concat_df


In [21]:
df_dict_initial.keys()

dict_keys(['df_601', 'df_602', 'df_603', 'df_604', 'df_605', 'df_606', 'df_607', 'df_608', 'df_609', 'df_610', 'df_611', 'df_612', 'df_613', 'df_614', 'df_615', 'df_616', 'df_617', 'df_618', 'df_619', 'df_620', 'df_621', 'df_622', 'df_623', 'df_624', 'df_625', 'df_626', 'df_627', 'df_628', 'df_629', 'df_630', 'df_631', 'df_632', 'df_633', 'df_634', 'df_635', 'df_636', 'df_637', 'df_639', 'df_641', 'df_642', 'df_643', 'df_644', 'df_645', 'df_646', 'df_647', 'df_648', 'df_649', 'df_650', 'df_651', 'df_652', 'df_653', 'df_654', 'df_655', 'df_656', 'df_657', 'df_658', 'df_659', 'df_660', 'df_661', 'df_662', 'df_663', 'df_664', 'df_665', 'df_666', 'df_667', 'df_668', 'df_669', 'df_670', 'df_672', 'df_673', 'df_674', 'df_675', 'df_676', 'df_677', 'df_678', 'df_679', 'df_680', 'df_681', 'df_682', 'df_683', 'df_684', 'df_685', 'df_686', 'df_687', 'df_688', 'df_689', 'df_690', 'df_691', 'df_692', 'df_693', 'df_694', 'df_695', 'df_696', 'df_697', 'df_698', 'df_699', 'df_700', 'df_701', 'df_702',

<div class="alert alert-block alert-warning">
  <p>
    <b>Evaluation of the results</b>
  </p>

  <p>
    

  </p>
    
  <p>
  </p>
</div>

<div class="alert alert-block alert-warning">
  <p>
    <b>Remark</b>
  </p>

  <p>
    The trt values does not, necessarly, correspond to a new series of data. In some cases many of the values are repeated.
    Lines of action:
      <ul>
            <li>Clean repeat information</li>
      </ul>

  </p>

</div>

<div class="alert alert-block alert-warning">

 <p>
    The following function does a comparision only when we are considering 2 DataFrames. When we are considering more than 2 DataFrames, it just merge them. For the nature of the data is very unlikley that 3 potentially DataFrames, or more, to merge are not corresponding to the same patient. Anyway after the merge we can compare the difference between the first timestamp of information and the last one. 
  </p>
</div>

In [22]:
'''Function to see if the potentially DataFrames are close enough, according to the timestamp, so we can merge them. days indicates the maximum tolerance 
allowed between DataFrames'''
days = 5
delete_elements=[]

for dframe in df_dict_initial.keys():
    if 'concat' in dframe:
        df_test = df_dict_initial[dframe]
        initial = df_test["Date__Heure"].iloc[0]
        final = df_test["Date__Heure"].iloc[-1]
        decision = abs(initial - final) <= pd.Timedelta(days=days)
        if decision == True:
            #Delete individual DataFrames
            base_part = dframe.replace('-concat', '')
            parts = base_part.split('-')
            for item in parts:
                delete_elements.append(item) 
        else:
            #Delete merge DataFrames
            delete_elements.append(dframe)

for element in delete_elements:
    del df_dict_initial[element] 

In [23]:
'''At this point df_dict_initial contains DataFrames corresponding to a single treatment in 
case the previous merged conditions were not satisfied or a merged DataFrame in the opposite case.
If the DataFrames were merged, the original ones were deleted'''
df_dict_initial.keys()

dict_keys(['df_601', 'df_602', 'df_603', 'df_604', 'df_605', 'df_606', 'df_607', 'df_608', 'df_609', 'df_610', 'df_611', 'df_612', 'df_613', 'df_614', 'df_615', 'df_616', 'df_617', 'df_620', 'df_621', 'df_622', 'df_623', 'df_626', 'df_627', 'df_628', 'df_629', 'df_630', 'df_631', 'df_632', 'df_633', 'df_634', 'df_635', 'df_636', 'df_637', 'df_639', 'df_641', 'df_642', 'df_643', 'df_644', 'df_645', 'df_646', 'df_647', 'df_648', 'df_649', 'df_650', 'df_651', 'df_652', 'df_653', 'df_654', 'df_655', 'df_656', 'df_657', 'df_658', 'df_659', 'df_662', 'df_663', 'df_664', 'df_665', 'df_666', 'df_667', 'df_668', 'df_669', 'df_670', 'df_672', 'df_673', 'df_674', 'df_675', 'df_676', 'df_677', 'df_678', 'df_679', 'df_689', 'df_690', 'df_691', 'df_692', 'df_693', 'df_694', 'df_695', 'df_696', 'df_697', 'df_700', 'df_701', 'df_702', 'df_703', 'df_704', 'df_705', 'df_706', 'df_707', 'df_708', 'df_709', 'df_710', 'df_711', 'df_714', 'df_715', 'df_716', 'df_717', 'df_725', 'df_726', 'df_727', 'df_728',

<div class="alert alert-block alert-warning">
  <p>
    <b>How to save this DataFrames?</b>
  </p>

  <p>
    <li>Will depend on how we are going to use them for the model deployment</li> 
    

  </p>

</div>

In [24]:
#For quality control, we can verify the first and the last row of the concat DataFrames and see if the difference is not bigger than 3 days

In [25]:
def quality_control(df_dict,days):
    '''The function is a quality control check to see if the difference between the intial and the last timestamp is not considerably big.
    We expect to have a difference of days because of the nature of the data.

    
    Parameters
    ----------
    df_dict : dict 
        dictionary in which each key is a DataFrame
    days : int
        tolerance in days

    
    Returns
    -------
    None : list
        the function prints 'Error + DataFrame name' i.e., 'Error df_601' if a DataFrame does not meet the given tolerance
        
    '''
    for dframe in df_dict.keys():
        df_test = df_dict[dframe]
        initial = df_test["Date__Heure"].iloc[0]
        final = df_test["Date__Heure"].iloc[-1]
        decision = abs(initial - final) <= pd.Timedelta(days=days)
        if decision == False:
            print(f'Error {dframe}')
        else:
            continue
    print('If no message was displayed before this, the quality control was passed')

In [26]:
#Let's try to save the final dictionary and see if we can open it in another notebook

In [27]:
file_no_ext = os.path.splitext(file)[0]
name_save = f'{path_to_save}/{file_no_ext}.pkl' 
with open(name_save, 'wb') as file:
    pickle.dump(df_dict_initial, file)

In [28]:
name_save

'/Users/luisescobar/Documents/Thesis/DataSets/Dictionary/completo_800_output_file.pkl'